# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f201823a250>
├── 'a' --> tensor([[-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071]])
└── 'x' --> <FastTreeValue 0x7f201823a760>
    └── 'c' --> tensor([[-0.6748,  0.5218,  0.4836,  0.3265],
                        [-0.0230, -1.6345, -0.2861,  0.4661],
                        [ 1.7503, -1.0361,  1.6852,  1.1124]])

In [4]:
t.a

tensor([[-1.9110, -2.0608, -1.4630],
        [ 1.0848, -0.4555,  0.4071]])

In [5]:
%timeit t.a

76.2 ns ± 0.642 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f201823a250>
├── 'a' --> tensor([[-0.1338,  1.3141, -1.8072],
│                   [ 0.3532,  0.0132,  0.2284]])
└── 'x' --> <FastTreeValue 0x7f201823a760>
    └── 'c' --> tensor([[-0.6748,  0.5218,  0.4836,  0.3265],
                        [-0.0230, -1.6345, -0.2861,  0.4661],
                        [ 1.7503, -1.0361,  1.6852,  1.1124]])

In [7]:
%timeit t.a = new_value

83.2 ns ± 1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.9110, -2.0608, -1.4630],
               [ 1.0848, -0.4555,  0.4071]]),
    x: Batch(
           c: tensor([[-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124]]),
       ),
)

In [10]:
b.a

tensor([[-1.9110, -2.0608, -1.4630],
        [ 1.0848, -0.4555,  0.4071]])

In [11]:
%timeit b.a

74.1 ns ± 1.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0574, -1.2533, -1.2332],
               [ 0.9505, -0.7311,  0.7362]]),
    x: Batch(
           c: tensor([[-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124]]),
       ),
)

In [13]:
%timeit b.a = new_value

662 ns ± 4.63 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 7.45 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

300 µs ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

293 µs ± 27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f201415b610>
├── 'a' --> tensor([[[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]],
│           
│                   [[-1.9110, -2.0608, -1.4630],
│                    [ 1.0848, -0.4555,  0.4071]]])
└── 'x' --> <FastTreeValue 0x7f1f3daa2790>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

52.1 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1f3dabe130>
├── 'a' --> tensor([[-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071],
│                   [-1.9110, -2.0608, -1.4630],
│                   [ 1.0848, -0.4555,  0.4071]])
└── 'x' --> <FastTreeValue 0x7f1f3dabee50>
    └── 'c' --> tensor([[-0.6748,  0.5218,  0.4836,  0.3265],
                        [-0.0230, -1.6345, -0.2861,  0.4661],
                 

In [23]:
%timeit t_cat(trees)

50.5 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

109 µs ± 4.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.6748,  0.5218,  0.4836,  0.3265],
                       [-0.0230, -1.6345, -0.2861,  0.4661],
                       [ 1.7503, -1.0361,  1.6852,  1.1124]],
              
                      [[-0.6748,  0.5218,  0.4836,  0.3265],
                       [-0.0230, -1.6345, -0.2861,  0.4661],
                       [ 1.7503, -1.0361,  1.6852,  1.1124]],
              
                      [[-0.6748,  0.5218,  0.4836,  0.3265],
                       [-0.0230, -1.6345, -0.2861,  0.4661],
                       [ 1.7503, -1.0361,  1.6852,  1.1124]],
              
                      [[-0.6748,  0.5218,  0.4836,  0.3265],
                       [-0.0230, -1.6345, -0.2861,  0.4661],
                       [ 1.7503, -1.0361,  1.6852,  1.1124]],
              
                      [[-0.6748,  0.5218,  0.4836,  0.3265],
                       [-0.0230, -1.6345, -0.2861,  0.4661],
                       [ 1.7503, -1.0361,  1.6852,  1.1124]],

In [26]:
%timeit Batch.stack(batches)

113 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124],
                      [-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124],
                      [-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124],
                      [-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124],
                      [-0.6748,  0.5218,  0.4836,  0.3265],
                      [-0.0230, -1.6345, -0.2861,  0.4661],
                      [ 1.7503, -1.0361,  1.6852,  1.1124],
                      [-0.6748,  0.5218,  0.4836,  0.3265],
                   

In [28]:
%timeit Batch.cat(batches)

215 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

678 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
